### Задание 1
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону.

Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона.

Если поисковый запрос не содержит названия города, то ставим 'undefined'. 

Результат классификации запишите в отдельный столбец region.

In [1]:
### Правила распределения по регионам Центр, Северо-Запад и Дальний Восток: ###
geo_data = { 'Центр':          ['москва',      'тула',    'ярославль'],
             'Северо-Запад':   ['петербург',   'псков',   'мурманск'], 
             'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'] }
import pandas as pd
keywords = pd.read_csv('keywords.csv')
keywords.head()
for x in geo_data.items():
    keywords['keyword'].replace(to_replace=list(x[1]), value=str(x[0]),inplace=True)
### Проверка, удаласьли замена ###
print(keywords.query('keyword == {}'.format(['Дальний Восток'])))

              keyword  shows
5858   Дальний Восток  43038
9084   Дальний Восток  29655
21445  Дальний Восток  14494


### Задание 2
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг
 
Результат классификации запишите в столбец class

In [2]:
ratings = pd.read_csv('ratings.csv')
def movies_classification(input0):
    rez=[input0.filter(items = ['movieId','rating']).groupby('movieId').agg('mean').query('rating <{}'.format([2])),\
         input0.filter(items = ['movieId','rating']).groupby('movieId').agg('mean').query('rating >={} & rating <{}'.format([2],[4.5])),\
         input0.filter(items = ['movieId','rating']).groupby('movieId').agg('mean').query('rating >={}'.format([4.5]))]
    return rez;

rez0=movies_classification(ratings)
print('Вот список фильмов с низким рейтингом:\n')
print(rez0[0].head())
print('Вот список фильмов с средним рейтингом:\n')
print(rez0[1].head())
print('Вот список фильмов с высоким рейтингом:\n')
print(rez0[2].head())

Вот список фильмов с низким рейтингом:

           rating
movieId          
87       1.666667
96       1.000000
137      1.000000
181      1.800000
439      1.000000
Вот список фильмов с средним рейтингом:

           rating
movieId          
1        3.872470
2        3.401869
3        3.161017
4        2.384615
5        3.267857
Вот список фильмов с высоким рейтингом:

         rating
movieId        
53        5.000
80        4.625
116       4.750
130       4.500
178       4.700


### Задание 3
Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов).

Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя.

Ответ дайте в днях.

In [3]:
buf=ratings.filter(items = ['userId','timestamp']).groupby('userId').agg(['count','mean']).timestamp
print('Среднее занчение времени "жизни" пользователей с достаточно большим числом оценок (>100) равна:\n' + str(buf.query('count >={}'.format([100])).filter(items = ['mean']).mean()))

Среднее занчение времени "жизни" пользователей с достаточно большим числом оценок (>100) равна:
mean    1.135359e+09
dtype: float64


### Задание 4
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
 
При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:
 
1. В переменную years запишите список из всех годов с 1950 по 2010.
 
2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
- для каждой строки пройдите по всем годам списка years
- если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
- если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
 
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'
 
4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [4]:
movies = pd.read_csv('movies.csv')
### создание списка рассматриваемых годов ###
years=[]
y=1950
while y<=2010:
    years.append(y)
    y=y+1
### функция возвращающая год по имени фильма ###
def production_year(film_name,years):
    Years0=1900
    for x in years:
        buf=str(x)
        if film_name.find(buf)>=0:
            Years0=x
            break
    return Years0

In [5]:
### заполнение столбца year значениями ###
movies['year'] = 0
for i in range(len(movies)):
    movies['year'][i]=production_year(movies['title'][i],years)

/home/solovushka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [6]:
rez=movies.join(ratings, lsuffix='movieId', rsuffix='movieId').groupby('year').agg('mean').rating
### вывод результата ###
rez.sort_values(ascending=False).head()

year
2007    4.021484
1952    3.851852
1994    3.833333
2008    3.819838
2006    3.808429
Name: rating, dtype: float64